# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
from collections import Counter
import pandas as pd
from pymystem3 import Mystem

data = pd.read_csv('/datasets/data.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

1.В столбцах 'days_employed' и 'total_income' есть пропущенные значение, что говорит о взаимосвязи отработанных дней и ежемесячном доходе
2.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data.isna().any()) # Посмотрел в каких столбцах есть значение Nan
print(data.isna().sum()) # Посмотрел их количество
data = data.fillna(data.groupby(['income_type']).transform('mean')) # Заполнил пустые данные на основе группировки по типу занятости


children            False
days_employed        True
dob_years           False
education           False
education_id        False
family_status       False
family_status_id    False
gender              False
income_type         False
debt                False
total_income         True
purpose             False
dtype: bool
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


**Вывод**

Заменил значение столбцов total_income и days_employed на ноль.

### Замена типа данных

In [3]:
data['days_employed']=abs(data.loc[:,'days_employed']) # взял модуль по столбцу
data['children'] = abs(data.loc[:,'children']) # взял модуль по столбцу

**Вывод**

Перевел отрицательные значение в столбцах children и days_employed в положительные с помощью модуля

### Обработка дубликатов

In [4]:
data['education'] = data['education'].str.lower() # Привел к нижнему регистру, чтобы исключить дубликаты
data = data.drop_duplicates()  # удалли дубликаты
print(data['gender'].value_counts()) #Вывел сгрупированные значения колонки gender
data = data[data['gender'] !='XNA'] #удалил значения пола = XNA
data = data[data['dob_years'] != 0]  # удалил строки, где возраст = 0
data = data[data['children']!=20] # удалил строки, где в семье 20 детей

F      14174
M       7279
XNA        1
Name: gender, dtype: int64


**Вывод**

Привет столбец education к нижнему регистру, чтобы убрать повторения. Так же удалил строку, где значение в столбце gender было XNA

### Лемматизация

In [5]:
m = Mystem()
def lem_purpose(purpose):
    lem = m.lemmatize(purpose)
    return lem
    
data['lem_purpose'] = data['purpose'].apply(lem_purpose)
print(Counter(data['lem_purpose'].sum()).most_common())
def lem_category(lem_purpose):    
    if 'образование' in lem_purpose:
        return 'На обзразование'
    if 'жилье' in lem_purpose or 'недвижимость' in lem_purpose:
        return 'На жилье'
    if 'автомобиль' in lem_purpose:
        return 'На автомобиль'
    if 'свадьба' in lem_purpose:
        return 'На свадьбу'
    return 'иное'
data['lem_category'] = data['lem_purpose'].apply(lem_category)
print(data['lem_category'].value_counts())
# data['lem_cat'] = data['purpose'].apply(lemma)
# print(data['lem_cat'].value_counts())

[(' ', 33312), ('\n', 21277), ('недвижимость', 6306), ('покупка', 5851), ('жилье', 4421), ('автомобиль', 4269), ('образование', 3980), ('с', 2891), ('операция', 2583), ('свадьба', 2301), ('свой', 2219), ('на', 2200), ('строительство', 1866), ('высокий', 1360), ('получение', 1306), ('коммерческий', 1299), ('для', 1284), ('жилой', 1221), ('сделка', 934), ('заниматься', 900), ('дополнительный', 898), ('проведение', 760), ('сыграть', 756), ('сдача', 648), ('семья', 636), ('собственный', 627), ('со', 626), ('ремонт', 603), ('подержанный', 480), ('подержать', 472), ('приобретение', 459), ('профильный', 433)]
На жилье           10727
На автомобиль       4269
На обзразование     3980
На свадьбу          2301
Name: lem_category, dtype: int64


**Вывод**

### Категоризация данных

In [6]:
def total_inc(total_income):
    if total_income <= 107000:
        return '0-107'
    if 107001 <= total_income <= 150000:
        return '107-150'
    if 150001 <= total_income <= 202000:
        return '150-202'
    return '202+'
data['income'] = data['total_income'].apply(total_inc)
print(data['income'].value_counts())

150-202    5421
202+       5390
0-107      5264
107-150    5202
Name: income, dtype: int64


**Вывод**

Разбил ежемесячный доход на категории с шагом в 40к рублей.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
data_children = data.pivot_table(index=['children'],values='debt',aggfunc=['sum','count','mean'])
data_children.columns = ['sum','total','%']
display(data_children.sort_values(by = 'sum',ascending=True))

,sum,total,%
children,,,
5,0,9,0.000000
4,4,41,0.097561
3,27,328,0.082317
2,194,2039,0.095145
1,442,4839,0.091341
0,1058,14021,0.075458


**Вывод**

Из результата видно, что при количество детей влияет на возврат кредита в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
data_family_status = data.pivot_table(index=['family_status'],values='debt',aggfunc=['sum','count','mean'])
data_family_status.columns = ['sum','total','%']
display(data_family_status.sort_values(by = 'sum',ascending=True))

,sum,total,%
family_status,,,
вдовец / вдова,62,950,0.065263
в разводе,84,1183,0.071006
Не женат / не замужем,272,2785,0.097666
гражданский брак,383,4117,0.093029
женат / замужем,924,12242,0.075478


**Вывод**

Из результата видно, что при наличии пары возрастает процент людей, которые не возвращают кредиты в срок

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
data_income = data.pivot_table(index=['income'],values='debt',aggfunc=['sum','count','mean'])
data_income.columns = ['sum','total','%']
display(data_income.sort_values(by = 'total',ascending=False))

,sum,total,%
income,,,
150-202,478,5421,0.088176
202+,375,5390,0.069573
0-107,422,5264,0.080167
107-150,450,5202,0.086505


**Вывод**

Из результата видно,что с увлечением ежемесячного дохода растер и процент людей, которые не возвращают кредиты в срок. Любопытно, что люди с достатком 170-210к имеют меньший процент.

<div class="alert alert-info"> Из резульатата видно, что процент по задолжностям у всех одинаковый </div>

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
data_lem_category = data.pivot_table(index=['lem_category'],values='debt',aggfunc=['count','mean'])
data_lem_category.columns = ['total','%']
display(data_lem_category.sort_values(by = 'total',ascending=False))

,total,%
lem_category,,
На жилье,10727,0.072434
На автомобиль,4269,0.093230
На обзразование,3980,0.092714
На свадьбу,2301,0.078661


**Вывод**

Как видно из таблицы больше всего должников  в категории "На свадьбу", остальных примерно поровну

<div class="alert alert-info"> Из резульатата видно, что процент должников равный  </div>

## Шаг 4. Общий вывод

В сводной таблице 
видно,что самые злостные нарушители те, у кого ежемесячный доход 150-202 не имеет детей и женат\замужем

In [11]:
data_pivot = data.pivot_table(index=['income','family_status'],columns = 'children',values='debt',aggfunc='sum')
display(data_pivot)

children                           0     1     2    3    4    5
income  family_status                                          
0-107   Не женат / не замужем   47.0   7.0   0.0  0.0  1.0  NaN
        в разводе               12.0   5.0   5.0  0.0  NaN  NaN
        вдовец / вдова          15.0   0.0   0.0  NaN  NaN  NaN
        гражданский брак        53.0  29.0  10.0  4.0  0.0  NaN
        женат / замужем        133.0  63.0  34.0  4.0  0.0  0.0
107-150 Не женат / не замужем   51.0  18.0   3.0  0.0  NaN  NaN
        в разводе               15.0   5.0   0.0  1.0  NaN  NaN
        вдовец / вдова          15.0   3.0   1.0  0.0  NaN  NaN
        гражданский брак        69.0  28.0   4.0  1.0  0.0  0.0
        женат / замужем        127.0  68.0  39.0  2.0  0.0  NaN
150-202 Не женат / не замужем   60.0  15.0   5.0  1.0  0.0  NaN
        в разводе               17.0   5.0   0.0  0.0  NaN  NaN
        вдовец / вдова          16.0   3.0   1.0  NaN  0.0  NaN
        гражданский брак        59.0  39.0   9.0  2.0  0.0  0.0
        женат / замужем        139.0  58.0  39.0  8.0  2.0  0.0
202+    Не женат / не замужем   52.0  11.0   1.0  0.0  NaN  NaN
        в разводе               11.0   6.0   2.0  0.0  0.0  NaN
        вдовец / вдова           6.0   1.0   1.0  0.0  NaN  NaN
        гражданский брак        46.0  22.0   7.0  1.0  0.0  NaN
        женат / замужем        115.0  56.0  33.0  3.0  1.0  0.0